In [2]:
# https://www.kaggle.com/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition
# Essentials
import pandas as pd
import numpy as np
import time
import sys
import math

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Models
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import train_test_split

# Ignorar ciertos warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000


from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn import decomposition, datasets
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

import math
from scipy.spatial import distance
from scipy.stats import ks_2samp
from scipy import spatial

In [3]:
import sklearn
sklearn.__version__

'0.22.1'

# Tarea de Minería

## Importing Data

In [4]:
# Data para entrenar el algoritmo de regresión
dataPesca = pd.read_csv("calaPesca.csv", sep=";")

dataPesca = dataPesca[["LONGI","LATIT","Salinidad","TC","Clorofila (mg/m3)","TSM (ºC)","Captura"]] #filtra solo lo que es igual al data frame "df"

dataPesca.columns = ["LONGI","LATIT","Salinidad","TC", "Clorofila","TSM","Captura" ]
dataPesca.head(3)

,LONGI,LATIT,Salinidad,TC,Clorofila,TSM,Captura
0,-78.12,-11.17,35.06,63,0.569,19.1,100
1,-78.18,-11.23,35.06,64,0.495,19.2,60
2,-78.38,-10.70,35.07,65,16.218,19.4,80


In [5]:
# Bucketización de la variable continua de pesca
dataPesca["CapturaBucketized"] = 999
dataPesca.loc[dataPesca.Captura <= 100, "CapturaBucketized"] = 1
dataPesca.loc[dataPesca.Captura > 100, "CapturaBucketized"] = 0

In [6]:
dataPesca.CapturaBucketized.value_counts()

1    6899
0    2630
Name: CapturaBucketized, dtype: int64

In [7]:
# Elimina outliers
dataPesca.drop(dataPesca[dataPesca['LONGI']>-40].index, inplace=True)
dataPesca.drop(dataPesca[dataPesca['TC']>85].index, inplace=True)
dataPesca.dropna(inplace=True)

## Modelamiento

In [8]:
#dataPesca = dataPesca.drop("CapturaBucketized", axis=1)

# Sacamos independientes y dependiente
Y = dataPesca["CapturaBucketized"]
X = dataPesca.drop("CapturaBucketized", axis=1)
X = X[['Salinidad', "TC", "Clorofila", "TSM"]]

# Dividimos en train, test en un 70 - 30
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state=42)

In [9]:
X.describe()

,Salinidad,TC,Clorofila,TSM
count,9527.000000,9527.000000,9527.000000,9527.000000
mean,33.823038,57.971450,3.125849,17.854298
std,6.477817,15.811708,5.040272,3.676209
min,0.000000,0.000000,0.000000,0.000000
25%,35.010000,51.000000,0.275000,17.300000
50%,35.060000,61.000000,1.396000,18.400000
75%,35.110000,69.000000,3.802000,19.500000
max,35.380000,75.000000,56.234000,23.900000


In [10]:
def cv_acc(model, X=X_train, kf = 3):
    acc = np.sqrt(-cross_val_score(model, X_train, y_train_enc, scoring="accuracy", cv=kf))
    return (acc)

In [11]:
kf = KFold(n_splits=3, shuffle=True)

In [12]:
std_slc = StandardScaler()
pca = decomposition.PCA()
dec_tree = tree.DecisionTreeClassifier()

In [13]:
dec_tree

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [14]:
dec_tree.criterion = 'entropy'

In [15]:
dec_tree

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

### Tuneo de hiperparámetros

In [16]:
pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('dec_tree', dec_tree)])

#n_components = list(range(1,X_train.shape[1]+1,1))

In [17]:
params = {}#dict(pca__n_components=n_components)

In [18]:
params['dec_tree__criterion'] = ['gini', 'entropy']
params['dec_tree__splitter'] = ['best','random']
params['dec_tree__min_samples_split'] = [0.01, 0.05, 0.1, 0.15, 0.16, 0.18, 0.2]
params["dec_tree__max_features"] = ["auto", "sqrt", "log2"] 
params['dec_tree__max_depth'] = [2,4,6,8,9,10,11,12,20,30,40,50]

In [19]:
params

{'dec_tree__criterion': ['gini', 'entropy'],
 'dec_tree__splitter': ['best', 'random'],
 'dec_tree__min_samples_split': [0.01, 0.05, 0.1, 0.15, 0.16, 0.18, 0.2],
 'dec_tree__max_features': ['auto', 'sqrt', 'log2'],
 'dec_tree__max_depth': [2, 4, 6, 8, 9, 10, 11, 12, 20, 30, 40, 50]}

In [20]:
decisionTree = GridSearchCV(pipe, param_grid = params, cv = kf, scoring='accuracy') 
decisionTree.fit(X_train, Y_train)
decisionTree.best_params_

{'dec_tree__criterion': 'gini',
 'dec_tree__max_depth': 10,
 'dec_tree__max_features': 'sqrt',
 'dec_tree__min_samples_split': 0.01,
 'dec_tree__splitter': 'random'}

In [21]:
decisionTree.best_score_

0.7236057613858552

### Set up the models

In [22]:
std_slc = StandardScaler()
dec_tree = tree.DecisionTreeClassifier(criterion='gini',max_depth=11, max_features = 'auto', min_samples_split = 0.01, splitter = 'random')

In [23]:
pipe = Pipeline(steps=[('std_slc', std_slc),
                       ('dec_tree', dec_tree)])

In [24]:
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('std_slc',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('dec_tree',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=11,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1,
                                        min_samples_split=0.01,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='random'))],
         verbose=False)

### Verifica los resultados del CV

In [25]:
from sklearn.metrics import accuracy_score
prd = pipe.predict(X_train)
accuracy_score(Y_train, prd)

0.7263047390521895

In [26]:
prd_test = pipe.predict(X_test)
accuracy_score(Y_test, prd_test)

0.7324239244491081

1    6898
0    2629

In [27]:
X_standarized = std_slc.transform(X)

In [28]:
Y.value_counts()

1    6898
0    2629
Name: CapturaBucketized, dtype: int64

In [29]:
one_distribution = X_standarized[Y[Y == 1].index]
one_distribution

array([[ 0.19101261,  0.31821189, -0.50654666,  0.33907689],
       [ 0.19101261,  0.3816374 , -0.52113881,  0.36625096],
       [ 0.192556  ,  0.44506291,  2.57929858,  0.42059912],
       ...,
       [ 0.19718618,  0.50848841,  1.8924816 ,  0.06733612],
       [ 0.18483903,  0.88904145, -0.48071461,  0.04016205],
       [ 0.19718618,  0.50848841,  1.8924816 ,  0.06733612]])

In [30]:
zero_distribution = X_standarized[Y[Y == 0].index[:-1]]
print(zero_distribution)


[[ 0.19409939  0.3816374   0.30351483  0.36625096]
 [ 0.19564279  0.3816374   0.9840217   0.39342504]
 [ 0.19564279  0.3816374   0.9840217   0.39342504]
 ...
 [ 0.20027297  0.95246696  1.8924816   0.71951396]
 [ 0.20181637  0.95246696  2.69169757  0.6651658 ]
 [ 0.20644655  1.01589247 -0.55860514  0.93690657]]


In [31]:
dfo = pd.DataFrame(one_distribution)
dfo.columns = ["Salinidad","TC", "Clorofila","TSM"]
dfo.describe()

,Salinidad,TC,Clorofila,TSM
count,6898.000000,6898.000000,6898.000000,6898.000000
mean,-0.040169,-0.096785,-0.014185,-0.043937
std,1.095034,1.052992,0.980216,1.081239
min,-5.220127,-3.677595,-0.618748,-4.851172
25%,0.181752,-0.506320,-0.618748,-0.177231
50%,0.191013,0.191361,-0.361808,0.094510
75%,0.198730,0.571914,0.130973,0.447773
max,0.238858,1.079318,10.093664,1.643433


In [32]:
dfz = pd.DataFrame(zero_distribution)
dfz.columns = ["Salinidad","TC", "Clorofila","TSM"]
dfz.describe()

,Salinidad,TC,Clorofila,TSM
count,2628.000000,2628.000000,2628.000000,2628.000000
mean,0.105732,0.251601,0.028919,0.117393
std,0.678991,0.804940,1.028593,0.728411
min,-5.220127,-3.677595,-0.618748,-4.851172
25%,0.183296,0.064510,-0.480715,-0.122882
50%,0.191013,0.254786,-0.302651,0.176032
75%,0.200273,0.968323,0.130973,0.502121
max,0.240401,1.079318,10.470102,1.616258


## Normal

In [33]:
import numpy as np

df_results = pd.DataFrame(columns=['Params','IL_EucDistance','IL_Cosimilitud','DR_JS','DR_KS'])
df_count = pd.DataFrame(columns=['One','Zero'])
j = 0

while (j < 10):
    
    #Salinidad', "TC", "Clorofila", "TSM
    Distillation_Salinidad = []
    Distillation_TC = []
    Distillation_Clorofila = []
    Distillation_TSM = []
    
    print ("j:",j)
    #class one
    Distillation_Salinidad.extend(np.random.normal(-0.040169 , 1.095034, 6898)) #Salinidad
    Distillation_TC.extend(np.random.normal(-0.096785 , 1.052992, 6898)) #TC
    Distillation_Clorofila.extend(np.random.normal(-0.014185 , 0.980216, 6898)) #Clorofila
    Distillation_TSM.extend(np.random.normal(-0.043937 , 1.081239, 6898)) #TSM

    #class zero
    Distillation_Salinidad.extend(np.random.normal( 0.105732, 0.678991, 2629)) #Salinidad
    Distillation_TC.extend(np.random.normal( 0.251601 , 0.804940, 2629)) #TC
    Distillation_Clorofila.extend(np.random.normal(0.028919  , 1.028593, 2629)) #Clorofila
    Distillation_TSM.extend(np.random.normal( 0.117393 , 0.728411 , 2629)) #TSM

    df_new = pd.DataFrame()
    df_new["Distillation_Salinidad"] = Distillation_Salinidad
    df_new["Distillation_TC"] = Distillation_TC
    df_new["Distillation_Clorofila"] = Distillation_Clorofila
    df_new["Distillation_TSM"] = Distillation_TSM

    df_new["PescaBucketizedNoisy"] = dec_tree.predict(df_new)

    #print (df_new.PescaBucketizedNoisy.value_counts())

    df_count = df_count.append({'One':int(df_new.PescaBucketizedNoisy.value_counts()[1]), 'Zero':int(df_new.PescaBucketizedNoisy.value_counts()[0])}, ignore_index=True)

    df_new["SALI"] = dataPesca["Salinidad"]
    df_new["TCL"] = dataPesca["TC"]
    df_new["CLO"] = dataPesca["Clorofila"]
    df_new["TSM"] = dataPesca["TSM"]

    import math

    # Get the Euclidean distance between vectors of real feature vs private vectors
    df_new["SquaredDifference"] = (df_new.Distillation_Salinidad - df_new.SALI)**2 + (df_new.Distillation_TC - df_new.TCL)**2 + (df_new.Distillation_Clorofila - df_new.CLO)**2 + (df_new.Distillation_TSM - df_new.TSM)**2
    df_new['EuclideanDistance'] = np.sqrt(df_new[['SquaredDifference']].sum(axis=1))

    # Cosimilitud
    r = []
    aux = 0
    for i in range(df_new.shape[0]):
        aux = spatial.distance.cosine(df_new.loc[i,["SALI","TCL","CLO","TSM"]], df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
        if (math.isnan(aux)):
            aux = spatial.distance.cosine(df_new.loc[i,["SALI","TCL","CLO","TSM"]]+0.00000001, df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
            if (math.isnan(aux)):
                aux = spatial.distance.cosine([0.00000001,0.00000001,0.00000001,0.00000001], df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
                #print (df_new.loc[i,["SALI","TCL","CLO","TSM"]]+0.00000001)
                #print (df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
                #print (aux)
        r.append(aux)

    # IL_EucDistance:
    IL_EucDistance = sum(df_new.EuclideanDistance)

    # IL_Cosimilitud:
    IL_Cosimilitud = sum(r)

    # DR Jensen Shannon: (1 - sum(abs(P_{verdadera}-V_{calculada})))/n 
    DR_JS = (1 - distance.jensenshannon(Y, df_new.PescaBucketizedNoisy))

    # DR Kolmogorov Smirnov
    # DR1: (1 - sum(P_{verdadera}-V_{calculada}))/n 
    DR_KS = (1 - ks_2samp(Y, df_new.PescaBucketizedNoisy)[0])

    # Resultados
    d = {'Params':["Distillation"],'IL_EucDistance': [IL_EucDistance], 'IL_Cosimilitud': [IL_Cosimilitud], 'DR_JS': [DR_JS], 'DR_KS':[DR_KS]}
    d = pd.DataFrame(data=d)

    df_results = df_results.append(d, ignore_index=True, sort=False)
    j += 1

j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
j: 7
j: 8
j: 9


In [34]:
df_count.describe()

,One,Zero
count,10,10
unique,10,10
top,5218,4320
freq,1,1


In [35]:
df_results.describe()

,IL_EucDistance,IL_Cosimilitud,DR_JS,DR_KS
count,10.000000,10.000000,10.000000,10.000000
mean,667418.788132,9418.752525,0.489909,0.819272
std,89.149469,46.889464,0.003008,0.005913
min,667263.188513,9340.412072,0.486093,0.807809
25%,667378.960968,9394.274676,0.487016,0.814895
50%,667415.336395,9409.880338,0.490198,0.822242
75%,667454.760526,9457.771412,0.492146,0.823292
max,667560.161161,9492.332830,0.494573,0.826493


In [36]:
df_results.to_csv(r'normal_entropy.csv', index = False)
df_count.to_csv(r'normal_entropy_count.csv', index = False)


## Uniform

In [37]:
import numpy as np

df_results = pd.DataFrame(columns=['Params','IL_EucDistance','IL_Cosimilitud','DR_JS','DR_KS'])
df_count = pd.DataFrame(columns=['One','Zero'])
j = 0

while (j < 10):
    
    #Salinidad', "TC", "Clorofila", "TSM
    Distillation_Salinidad = []
    Distillation_TC = []
    Distillation_Clorofila = []
    Distillation_TSM = []
    
    print ("j:",j)
    #class one
    Distillation_Salinidad.extend(np.random.uniform(-5.220127 , 0.238858, 6898)) #Salinidad
    Distillation_TC.extend(np.random.uniform(-3.677595 , 1.079318, 6898)) #TC
    Distillation_Clorofila.extend(np.random.uniform(-0.618748 , 10.093664, 6898)) #Clorofila
    Distillation_TSM.extend(np.random.uniform(-4.851172 , 1.643433, 6898)) #TSM

    #class zero
    Distillation_Salinidad.extend(np.random.uniform(-5.220127 , 0.240401, 2629)) #Salinidad
    Distillation_TC.extend(np.random.uniform(-3.677595 , 1.079318, 2629)) #TC
    Distillation_Clorofila.extend(np.random.uniform(-0.618748 , 10.470102, 2629)) #Clorofila
    Distillation_TSM.extend(np.random.uniform(-4.851172 , 1.616258, 2629)) #TSM

    df_new = pd.DataFrame()
    df_new["Distillation_Salinidad"] = Distillation_Salinidad
    df_new["Distillation_TC"] = Distillation_TC
    df_new["Distillation_Clorofila"] = Distillation_Clorofila
    df_new["Distillation_TSM"] = Distillation_TSM

    df_new["PescaBucketizedNoisy"] = dec_tree.predict(df_new)

    #print (df_new.PescaBucketizedNoisy.value_counts())

    df_count = df_count.append({'One':int(df_new.PescaBucketizedNoisy.value_counts()[1]), 'Zero':int(df_new.PescaBucketizedNoisy.value_counts()[0])}, ignore_index=True)

    df_new["SALI"] = dataPesca["Salinidad"]
    df_new["TCL"] = dataPesca["TC"]
    df_new["CLO"] = dataPesca["Clorofila"]
    df_new["TSM"] = dataPesca["TSM"]

    import math

    # Get the Euclidean distance between vectors of real feature vs private vectors
    df_new["SquaredDifference"] = (df_new.Distillation_Salinidad - df_new.SALI)**2 + (df_new.Distillation_TC - df_new.TCL)**2 + (df_new.Distillation_Clorofila - df_new.CLO)**2 + (df_new.Distillation_TSM - df_new.TSM)**2
    df_new['EuclideanDistance'] = np.sqrt(df_new[['SquaredDifference']].sum(axis=1))

    # Cosimilitud
    r = []
    aux = 0
    for i in range(df_new.shape[0]):
        aux = spatial.distance.cosine(df_new.loc[i,["SALI","TCL","CLO","TSM"]], df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
        if (math.isnan(aux)):
            aux = spatial.distance.cosine(df_new.loc[i,["SALI","TCL","CLO","TSM"]]+0.00000001, df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
            if (math.isnan(aux)):
                aux = spatial.distance.cosine([0.00000001,0.00000001,0.00000001,0.00000001], df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
                #print (df_new.loc[i,["SALI","TCL","CLO","TSM"]]+0.00000001)
                #print (df_new.loc[i,["Distillation_Salinidad","Distillation_TC","Distillation_Clorofila","Distillation_TSM"]])
                #print (aux)
        r.append(aux)

    # IL_EucDistance:
    IL_EucDistance = sum(df_new.EuclideanDistance)

    # IL_Cosimilitud:
    IL_Cosimilitud = sum(r)

    # DR Jensen Shannon: (1 - sum(abs(P_{verdadera}-V_{calculada})))/n 
    DR_JS = (1 - distance.jensenshannon(Y, df_new.PescaBucketizedNoisy))

    # DR Kolmogorov Smirnov
    # DR1: (1 - sum(P_{verdadera}-V_{calculada}))/n 
    DR_KS = (1 - ks_2samp(Y, df_new.PescaBucketizedNoisy)[0])

    # Resultados
    d = {'Params':["Distillation"],'IL_EucDistance': [IL_EucDistance], 'IL_Cosimilitud': [IL_Cosimilitud], 'DR_JS': [DR_JS], 'DR_KS':[DR_KS]}
    d = pd.DataFrame(data=d)

    df_results = df_results.append(d, ignore_index=True, sort=False)
    j += 1

j: 0
j: 1
j: 2
j: 3
j: 4
j: 5
j: 6
j: 7
j: 8
j: 9


In [38]:
df_results.to_csv(r'uniform_entropy.csv', index = False)
df_count.to_csv(r'uniform_entropy_count.csv', index = False)


In [39]:
df_results

,Params,IL_EucDistance,IL_Cosimilitud,DR_JS,DR_KS
0,Distillation,694473.864532,13247.004503,0.598626,0.908156
1,Distillation,694261.346288,13196.890616,0.603236,0.906161
2,Distillation,694481.808863,13255.131040,0.603150,0.904272
3,Distillation,694631.258137,13271.324909,0.603304,0.895350
4,Distillation,694618.370496,13233.725121,0.601198,0.898499
5,Distillation,694707.587804,13301.117192,0.597254,0.906266
6,Distillation,694593.280249,13294.445440,0.595168,0.912564
7,Distillation,694602.297255,13256.729521,0.601886,0.905217
8,Distillation,694509.202013,13237.024251,0.604140,0.899024
9,Distillation,694500.180293,13246.287790,0.599968,0.908261
